# model
rank1  Salesforce/SFR-Embedding-Mistral   ---right pad  (14344, 4096)

rank3 GritLM/GritLM-7B                    ---left pad ((14344, 4096))

rank4 intfloat/e5-mistral-7b-instruct     ---left pad   (14344, 4096)

rank8 jspringer/echo-mistral-7b-instruct-lasttoken (14344, 4096)

In [1]:
import numpy as np
import os
import os.path as op
# model_path = "jspringer/echo-mistral-7b-instruct-lasttoken"





In [4]:
# save_path = 'item_info'
# embeds_name = model_path.split('/')[-1]
# embeds_name = "item_cf_embeds_" + embeds_name + "_array.npy"
# data = np.load(op.join(save_path, embeds_name))
# norms = np.linalg.norm(data, ord=2, axis=1, keepdims=True)
# normalized_data = data / norms
# normalized_data
# squared_sums = np.sum(normalized_data**2, axis=1)
# normalized_data.shape


In [5]:
# embeds_name = model_path.split('/')[-1]
# embeds_name = "item_cf_embeds_Norm_" + embeds_name + "_array.npy"
# np.save(op.join(save_path, embeds_name), normalized_data)

In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch import Tensor
import numpy as np
import os
import os.path as op
from tqdm import tqdm

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModel

print(os.environ["HF_HOME"])
# print(os.environ["HF_HUB_CACHE"])


os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

/storage/lhsheng/miniconda3/envs/recint/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/storage_fast/lhsheng/huggingface_data/


In [2]:
model_path = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, device_map = 'auto')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
for param in model.parameters():
    param.requires_grad = False
# tokenizer.padding_side = "left"
# tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.24s/it]


In [3]:

item_df = pd.read_csv('raw_data/items_filtered.csv', index_col=0)
item_df.rename(columns={'title': 'item_name'}, inplace=True)
item_df

,item_id,item_name
0,0,The Prophet
1,1,Water For Elephants
2,2,For Whom the Bell Tolls
3,3,City of Djinns: A Year in Delhi
4,4,The Winds of War
...,...,...
40518,40518,Thumbprint
40519,40519,Going Nowhere Fast
40520,40520,In for a Penny (Five Star Mystery Series)
40521,40521,Dead Man Falling


In [4]:
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:# 向左边填充了所有的值。
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]
batch_size = 64
max_length = 128

for i in tqdm(range(0,len(item_df),batch_size)):
    item_names = item_df['item_name'][i:i+batch_size]
    # 生成输出
    batch_dict = tokenizer(item_names.tolist(), max_length=max_length, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**batch_dict)
    embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = embeddings.detach().cpu().numpy()
    if i == 0:
        item_embeds = embeddings
    else:
        item_embeds = np.concatenate((item_embeds, embeddings), axis=0)




  0%|          | 0/634 [00:00<?, ?it/s]

100%|██████████| 634/634 [24:55<00:00,  2.36s/it]


In [5]:
save_path = 'item_info'
embeds_name = model_path.split('/')[-1]
embeds_name = "item_cf_embeds_" + embeds_name + "_array.npy"
embeds_name, item_embeds.shape, op.join(save_path, embeds_name)

('item_cf_embeds_Mistral-7B-v0.1_array.npy',
 (40523, 4096),
 'item_info/item_cf_embeds_Mistral-7B-v0.1_array.npy')

In [6]:

np.save(op.join(save_path, embeds_name),item_embeds)

In [7]:
save_path = 'item_info'
embeds_name = model_path.split('/')[-1]
embeds_name = "item_cf_embeds_" + embeds_name + "_array.npy"
data = np.load(op.join(save_path, embeds_name))
norms = np.linalg.norm(data, ord=2, axis=1, keepdims=True)
normalized_data = data / norms
normalized_data
squared_sums = np.sum(normalized_data**2, axis=1)
squared_sums

array([0.9999999 , 1.        , 1.        , ..., 0.99999994, 0.99999994,
       0.99999994], dtype=float32)

In [8]:
embeds_name = model_path.split('/')[-1]
embeds_name = "item_cf_embeds_Norm_" + embeds_name + "_array.npy"
np.save(op.join(save_path, embeds_name), normalized_data)

: 